# Per-Target Ensemble Weight Optimization (exp_059)

**Hypothesis**: Different targets may benefit from different model weights.

**Rationale**:
- Current weights: GP(0.15) + MLP(0.55) + LGBM(0.30) for ALL targets
- SM is typically hardest to predict (highest variance)
- Product 2/3 may benefit from different weights
- Optimizing per-target weights can reduce overall MSE

**Implementation**:
1. Train GP, MLP, LGBM separately (get individual predictions)
2. Grid search over weight combinations for each target
3. Find optimal weights for SM, Product 2, Product 3 separately
4. Use optimized weights in final ensemble

**Best baseline**: exp_032 CV 0.008194 (GP 0.15 + MLP 0.55 + LGBM 0.30)

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = (X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"])
        yield (X[~mask], Y[~mask]), (X[mask], Y[mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer (for MLP and LGBM) - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip), dtype=torch.double)

# Simple Featurizer (for GP) - 18 features (Spange + Arrhenius kinetics)
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')
print(f'Simple feature dimension: {SimpleFeaturizer().feats_dim}')

Full feature dimension: 145
Simple feature dimension: 18


In [5]:
# MLP Model
class MLPModelInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3, dropout=0.05):
        super(MLPModelInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h_dim), nn.BatchNorm1d(h_dim), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h_dim
        layers.extend([nn.Linear(prev_dim, output_dim), nn.Sigmoid()])
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

print('MLP model defined')

MLP model defined


In [6]:
# Per-Target Weight Ensemble Model
class PerTargetWeightEnsemble:
    """Ensemble that uses different weights for each target."""
    def __init__(self, data='single', 
                 weights_product2=(0.15, 0.55, 0.30),  # (gp, mlp, lgbm)
                 weights_product3=(0.15, 0.55, 0.30),
                 weights_sm=(0.15, 0.55, 0.30)):
        self.data = data
        self.mixed = (data == 'full')
        self.weights_product2 = weights_product2
        self.weights_product3 = weights_product3
        self.weights_sm = weights_sm
        
        self.full_featurizer = FullFeaturizer(mixed=self.mixed)
        self.simple_featurizer = SimpleFeaturizer(mixed=self.mixed)
        
        # Models
        self.gp_models = None
        self.mlp_model = None
        self.lgbm_models = None
        self.scaler_full = None
        self.scaler_simple = None
        
    def train_model(self, X, Y):
        # Prepare features
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        Y_np = Y.values
        
        # Data augmentation for mixtures
        if self.mixed:
            X_full_flip = self.full_featurizer.featurize(X, flip=True)
            X_simple_flip = self.simple_featurizer.featurize(X, flip=True)
            X_full_all = np.vstack([X_full, X_full_flip])
            X_simple_all = np.vstack([X_simple, X_simple_flip])
            Y_all = np.vstack([Y_np, Y_np])
        else:
            X_full_all = X_full
            X_simple_all = X_simple
            Y_all = Y_np
        
        # Scale features
        self.scaler_full = StandardScaler()
        self.scaler_simple = StandardScaler()
        X_full_scaled = self.scaler_full.fit_transform(X_full_all)
        X_simple_scaled = self.scaler_simple.fit_transform(X_simple_all)
        
        # Train GP (3 models, one per target)
        self.gp_models = []
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        for i in range(3):
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=2, random_state=42+i)
            gp.fit(X_simple_scaled, Y_all[:, i])
            self.gp_models.append(gp)
        
        # Train MLP
        self.mlp_model = MLPModelInternal(input_dim=X_full_scaled.shape[1], hidden_dims=[32, 16], dropout=0.05).double().to(device)
        optimizer = torch.optim.Adam(self.mlp_model.parameters(), lr=5e-4, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20)
        loss_fn = nn.HuberLoss()
        
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y_all, dtype=torch.double).to(device)
        dataset = TensorDataset(X_tensor, Y_tensor)
        loader = DataLoader(dataset, batch_size=32, shuffle=True)
        
        self.mlp_model.train()
        for epoch in range(200):
            epoch_loss = 0
            for xb, yb in loader:
                optimizer.zero_grad()
                pred = self.mlp_model(xb)
                loss = loss_fn(pred, yb)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            scheduler.step(epoch_loss)
        self.mlp_model.eval()
        
        # Train LGBM (3 models, one per target)
        self.lgbm_models = []
        params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
                  'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9,
                  'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'seed': 42}
        for i in range(3):
            train_data = lgb.Dataset(X_full_scaled, label=Y_all[:, i])
            model = lgb.train(params, train_data, num_boost_round=100)
            self.lgbm_models.append(model)
        
    def predict(self, X):
        # Prepare features
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        if self.mixed:
            X_full_flip = self.full_featurizer.featurize(X, flip=True)
            X_simple_flip = self.simple_featurizer.featurize(X, flip=True)
        
        X_full_scaled = self.scaler_full.transform(X_full)
        X_simple_scaled = self.scaler_simple.transform(X_simple)
        
        if self.mixed:
            X_full_flip_scaled = self.scaler_full.transform(X_full_flip)
            X_simple_flip_scaled = self.scaler_simple.transform(X_simple_flip)
        
        # GP predictions
        gp_preds = np.column_stack([gp.predict(X_simple_scaled) for gp in self.gp_models])
        if self.mixed:
            gp_preds_flip = np.column_stack([gp.predict(X_simple_flip_scaled) for gp in self.gp_models])
            gp_preds = (gp_preds + gp_preds_flip) / 2
        
        # MLP predictions
        with torch.no_grad():
            mlp_preds = self.mlp_model(torch.tensor(X_full_scaled, dtype=torch.double).to(device)).cpu().numpy()
            if self.mixed:
                mlp_preds_flip = self.mlp_model(torch.tensor(X_full_flip_scaled, dtype=torch.double).to(device)).cpu().numpy()
                mlp_preds = (mlp_preds + mlp_preds_flip) / 2
        
        # LGBM predictions
        lgbm_preds = np.column_stack([m.predict(X_full_scaled) for m in self.lgbm_models])
        if self.mixed:
            lgbm_preds_flip = np.column_stack([m.predict(X_full_flip_scaled) for m in self.lgbm_models])
            lgbm_preds = (lgbm_preds + lgbm_preds_flip) / 2
        
        # Per-target weighted ensemble
        # Target order: Product 2, Product 3, SM (indices 0, 1, 2)
        combined = np.zeros_like(gp_preds)
        
        # Product 2 (index 0)
        w = self.weights_product2
        combined[:, 0] = w[0] * gp_preds[:, 0] + w[1] * mlp_preds[:, 0] + w[2] * lgbm_preds[:, 0]
        
        # Product 3 (index 1)
        w = self.weights_product3
        combined[:, 1] = w[0] * gp_preds[:, 1] + w[1] * mlp_preds[:, 1] + w[2] * lgbm_preds[:, 1]
        
        # SM (index 2)
        w = self.weights_sm
        combined[:, 2] = w[0] * gp_preds[:, 2] + w[1] * mlp_preds[:, 2] + w[2] * lgbm_preds[:, 2]
        
        return torch.clamp(torch.tensor(combined), 0, 1)

print('PerTargetWeightEnsemble defined')

PerTargetWeightEnsemble defined


In [7]:
# First, let's collect individual model predictions to find optimal weights
# We'll run CV and store GP, MLP, LGBM predictions separately

print('=== Collecting Individual Model Predictions for Weight Optimization ===')
X_single, Y_single = load_data("single_solvent")

# Store predictions from each model type
all_gp_preds = []
all_mlp_preds = []
all_lgbm_preds = []
all_actuals = []

split_generator = generate_leave_one_out_splits(X_single, Y_single)

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    # Prepare features
    full_featurizer = FullFeaturizer(mixed=False)
    simple_featurizer = SimpleFeaturizer(mixed=False)
    
    X_full = full_featurizer.featurize(train_X)
    X_simple = simple_featurizer.featurize(train_X)
    Y_np = train_Y.values
    
    scaler_full = StandardScaler()
    scaler_simple = StandardScaler()
    X_full_scaled = scaler_full.fit_transform(X_full)
    X_simple_scaled = scaler_simple.fit_transform(X_simple)
    
    # Train GP
    gp_models = []
    kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
    for i in range(3):
        gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=2, random_state=42+i)
        gp.fit(X_simple_scaled, Y_np[:, i])
        gp_models.append(gp)
    
    # Train MLP
    mlp = MLPModelInternal(input_dim=X_full_scaled.shape[1], hidden_dims=[32, 16], dropout=0.05).double().to(device)
    optimizer = torch.optim.Adam(mlp.parameters(), lr=5e-4, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20)
    loss_fn = nn.HuberLoss()
    
    X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
    Y_tensor = torch.tensor(Y_np, dtype=torch.double).to(device)
    dataset = TensorDataset(X_tensor, Y_tensor)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    mlp.train()
    for epoch in range(200):
        epoch_loss = 0
        for xb, yb in loader:
            optimizer.zero_grad()
            pred = mlp(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        scheduler.step(epoch_loss)
    mlp.eval()
    
    # Train LGBM
    lgbm_models = []
    params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
              'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9,
              'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'seed': 42}
    for i in range(3):
        train_data = lgb.Dataset(X_full_scaled, label=Y_np[:, i])
        model = lgb.train(params, train_data, num_boost_round=100)
        lgbm_models.append(model)
    
    # Get predictions on test set
    X_test_full = full_featurizer.featurize(test_X)
    X_test_simple = simple_featurizer.featurize(test_X)
    X_test_full_scaled = scaler_full.transform(X_test_full)
    X_test_simple_scaled = scaler_simple.transform(X_test_simple)
    
    gp_pred = np.column_stack([gp.predict(X_test_simple_scaled) for gp in gp_models])
    with torch.no_grad():
        mlp_pred = mlp(torch.tensor(X_test_full_scaled, dtype=torch.double).to(device)).cpu().numpy()
    lgbm_pred = np.column_stack([m.predict(X_test_full_scaled) for m in lgbm_models])
    
    all_gp_preds.append(gp_pred)
    all_mlp_preds.append(mlp_pred)
    all_lgbm_preds.append(lgbm_pred)
    all_actuals.append(test_Y.values)

# Stack all predictions
gp_preds = np.vstack(all_gp_preds)
mlp_preds = np.vstack(all_mlp_preds)
lgbm_preds = np.vstack(all_lgbm_preds)
actuals = np.vstack(all_actuals)

print(f'Collected predictions: GP {gp_preds.shape}, MLP {mlp_preds.shape}, LGBM {lgbm_preds.shape}')
print(f'Actuals: {actuals.shape}')

=== Collecting Individual Model Predictions for Weight Optimization ===


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:26<09:58, 26.02s/it]

  8%|▊         | 2/24 [00:48<08:52, 24.19s/it]

 12%|█▎        | 3/24 [01:09<07:56, 22.68s/it]

...

 92%|█████████▏| 22/24 [08:59<00:48, 24.47s/it]

 96%|█████████▌| 23/24 [09:22<00:24, 24.05s/it]

100%|██████████| 24/24 [09:47<00:00, 24.32s/it]

100%|██████████| 24/24 [09:47<00:00, 24.47s/it]

Collected predictions: GP (656, 3), MLP (656, 3), LGBM (656, 3)
Actuals: (656, 3)


In [ ]:
# Grid search for optimal weights per target
print('\n=== Grid Search for Optimal Weights per Target ===')

# Target names: Product 2 (idx 0), Product 3 (idx 1), SM (idx 2)
target_names = ['Product 2', 'Product 3', 'SM']

# Grid search over weights (must sum to 1)
weight_options = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

best_weights = {}
best_mse = {}

for target_idx, target_name in enumerate(target_names):
    print(f'\nOptimizing weights for {target_name}...')
    
    gp_target = gp_preds[:, target_idx]
    mlp_target = mlp_preds[:, target_idx]
    lgbm_target = lgbm_preds[:, target_idx]
    actual_target = actuals[:, target_idx]
    
    best_mse_target = float('inf')
    best_weights_target = (0.15, 0.55, 0.30)  # Default
    
    for w_gp in weight_options:
        for w_mlp in weight_options:
            w_lgbm = 1.0 - w_gp - w_mlp
            if w_lgbm < -0.01 or w_lgbm > 1.01:  # Allow small floating point errors
                continue
            w_lgbm = max(0, min(1, w_lgbm))  # Clamp to [0, 1]
            
            # Compute weighted prediction
            pred = w_gp * gp_target + w_mlp * mlp_target + w_lgbm * lgbm_target
            pred = np.clip(pred, 0, 1)
            
            # Compute MSE
            mse = np.mean((pred - actual_target) ** 2)
            
            if mse < best_mse_target:
                best_mse_target = mse
                best_weights_target = (w_gp, w_mlp, w_lgbm)
    
    best_weights[target_name] = best_weights_target
    best_mse[target_name] = best_mse_target
    print(f'  Best weights: GP={best_weights_target[0]:.2f}, MLP={best_weights_target[1]:.2f}, LGBM={best_weights_target[2]:.2f}')
    print(f'  Best MSE: {best_mse_target:.6f}')

# Compare with uniform weights
print('\n=== Comparison with Uniform Weights (0.15, 0.55, 0.30) ===')
for target_idx, target_name in enumerate(target_names):
    gp_target = gp_preds[:, target_idx]
    mlp_target = mlp_preds[:, target_idx]
    lgbm_target = lgbm_preds[:, target_idx]
    actual_target = actuals[:, target_idx]
    
    pred_uniform = 0.15 * gp_target + 0.55 * mlp_target + 0.30 * lgbm_target
    pred_uniform = np.clip(pred_uniform, 0, 1)
    mse_uniform = np.mean((pred_uniform - actual_target) ** 2)
    
    improvement = (mse_uniform - best_mse[target_name]) / mse_uniform * 100
    print(f'{target_name}: Uniform MSE={mse_uniform:.6f}, Optimized MSE={best_mse[target_name]:.6f}, Improvement={improvement:.2f}%')

In [ ]:
# Calculate overall MSE with optimized weights
print('\n=== Overall MSE Comparison ===')

# Uniform weights
pred_uniform = 0.15 * gp_preds + 0.55 * mlp_preds + 0.30 * lgbm_preds
pred_uniform = np.clip(pred_uniform, 0, 1)
mse_uniform = np.mean((pred_uniform - actuals) ** 2)

# Optimized per-target weights
pred_optimized = np.zeros_like(gp_preds)
for target_idx, target_name in enumerate(target_names):
    w = best_weights[target_name]
    pred_optimized[:, target_idx] = w[0] * gp_preds[:, target_idx] + w[1] * mlp_preds[:, target_idx] + w[2] * lgbm_preds[:, target_idx]
pred_optimized = np.clip(pred_optimized, 0, 1)
mse_optimized = np.mean((pred_optimized - actuals) ** 2)

print(f'Uniform weights (0.15, 0.55, 0.30): MSE = {mse_uniform:.6f}')
print(f'Optimized per-target weights: MSE = {mse_optimized:.6f}')
print(f'Improvement: {(mse_uniform - mse_optimized) / mse_uniform * 100:.2f}%')

print('\nOptimal weights found:')
for target_name in target_names:
    w = best_weights[target_name]
    print(f'  {target_name}: GP={w[0]:.2f}, MLP={w[1]:.2f}, LGBM={w[2]:.2f}')

In [ ]:
# Now run full CV with optimized weights
print('\n=== Full CV with Optimized Per-Target Weights ===')

# Extract optimal weights
w_product2 = best_weights['Product 2']
w_product3 = best_weights['Product 3']
w_sm = best_weights['SM']

print(f'Using weights:')
print(f'  Product 2: GP={w_product2[0]:.2f}, MLP={w_product2[1]:.2f}, LGBM={w_product2[2]:.2f}')
print(f'  Product 3: GP={w_product3[0]:.2f}, MLP={w_product3[1]:.2f}, LGBM={w_product3[2]:.2f}')
print(f'  SM: GP={w_sm[0]:.2f}, MLP={w_sm[1]:.2f}, LGBM={w_sm[2]:.2f}')

# Single solvent CV
X_single, Y_single = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X_single, Y_single)
all_predictions_single = []
all_actuals_single = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = PerTargetWeightEnsemble(data='single', 
                                     weights_product2=w_product2,
                                     weights_product3=w_product3,
                                     weights_sm=w_sm)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_single.append(predictions.numpy())
    all_actuals_single.append(test_Y.values)

preds_single = np.vstack(all_predictions_single)
actuals_single = np.vstack(all_actuals_single)
mse_single = np.mean((preds_single - actuals_single) ** 2)
print(f'Single Solvent MSE: {mse_single:.6f} (n={len(preds_single)})')

In [ ]:
# Full data CV with optimized weights
print('\n=== Full Data CV with Optimized Per-Target Weights ===')
X_full, Y_full = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = PerTargetWeightEnsemble(data='full', 
                                     weights_product2=w_product2,
                                     weights_product3=w_product3,
                                     weights_sm=w_sm)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_full.append(predictions.numpy())
    all_actuals_full.append(test_Y.values)

preds_full = np.vstack(all_predictions_full)
actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((preds_full - actuals_full) ** 2)
print(f'Full Data MSE: {mse_full:.6f} (n={len(preds_full)})')

In [ ]:
# Calculate overall MSE
n_single = len(preds_single)
n_full = len(preds_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE SUMMARY (Per-Target Weight Optimization) ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_032): 0.008194')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
    print(f'Predicted LB: {4.23 * overall_mse + 0.0533:.4f}')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')

In [ ]:
# Create submission from CV predictions
submission_single = []
for fold_idx, preds in enumerate(all_predictions_single):
    for row_idx, row in enumerate(preds):
        submission_single.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })
submission_single_solvent = pd.DataFrame(submission_single)

submission_full = []
for fold_idx, preds in enumerate(all_predictions_full):
    for row_idx, row in enumerate(preds):
        submission_full.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })
submission_full_data = pd.DataFrame(submission_full)

# Combine and save
submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

print(f'Submission saved: {len(submission)} rows')
print(f'Single solvent: {len(submission_single_solvent)}, Full data: {len(submission_full_data)}')

In [ ]:
# Final summary
print(f'\n=== FINAL SUMMARY ===')
print(f'\nOptimal weights found:')
for target_name in target_names:
    w = best_weights[target_name]
    print(f'  {target_name}: GP={w[0]:.2f}, MLP={w[1]:.2f}, LGBM={w[2]:.2f}')

print(f'\nCV Results:')
print(f'  Single Solvent MSE: {mse_single:.6f}')
print(f'  Full Data MSE: {mse_full:.6f}')
print(f'  Overall MSE: {overall_mse:.6f}')
print(f'\nBaseline (exp_032): CV 0.008194')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')